In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os

In [ ]:
os.environ["GOOGLE_API_KEY"] = " "

llm = ChatGoogleGenerativeAI(model = "gemini-pro")

In [3]:
conversation = """
Hi Sneh, how are you feeling today?

Hi, Doctor. I still have a high fever, cold, and cough. It's been like this for the past two days.

I see. Based on the symptoms you've described—high fever, cold, and cough—it sounds like you might have a viral infection, such as the flu or a common cold. These infections are quite common and usually resolve on their own with proper rest and medication. However, it’s important to monitor the fever and ensure it doesn’t get too high.

That makes sense. What can I do to feel better?

Continue taking Paracetamol for the fever. It's good to hear that it has been effective for you. I'll prescribe a few more medications to help with your symptoms. Take Paracetamol 500mg every 6 hours as needed for fever, Cetirizine 10mg before bed for your cold symptoms, and Guaifenesin 200mg after breakfast and after dinner for the cough.

Make sure to drink plenty of fluids and get plenty of rest.

Got it. How long should I continue taking these medications?

Take the medications for the next three days. If your symptoms persist or worsen, come back sooner. Otherwise, I would like you to come back for a follow-up check-up in five days to make sure you are recovering well.

Okay, I will do that. Thank you, Doctor.

You're welcome, Sneh. Take care and get well soon. Don't hesitate to reach out if you have any concerns before your follow-up.
"""

In [4]:
general_information = "Patient Name: Sneh Age: 20 Symptoms: - High fever (99 degrees Fahrenheit) - Cold - Cough Pain Assessment: None provided Medication: Paracetamol, effective for a few hours Symptom Duration: 2 days Additional Information: None"

In [5]:
from crewai import Crew, Agent, Task
from datetime import datetime

llm_agent = Agent(
    role="Patient Support Agent",
    goal="To provide post-medical consultation support to patients.",
    backstory="Trained to assist patients in managing their health after medical consultations.",
    llm=llm,
    tools=[],
    allow_delegation=False,
    verbose=True
)

# Define the task for the LLM agent
support_task = Task(
    description=(
        f"""
        Here is patient general information: {general_information}\n
        Here is the conversation between the patient and the doctor: {conversation}\n\n
        Provide recommendations for recovery, medication details, and schedule a follow-up check-up based on the patient details and conversation between patient and doctor.
        Give schedules accoding to: Patient breakfast time is 8:00 AM, lunch time is 12:00 PM, dinner time is 8:00 PM and Bed time is 10:00 PM. and today's date is {datetime.now().strftime('%Y-%m-%d')}
        After you get the necessary information parse it in json format where data should be in plain text.
        Only return the json data with no additional text
        keys of the json should be: 'recovery_recommendations', 'medication_details', 'follow_up_date'
        recover_recommendations should be a list of strings, medication_details should be a list of dictionaries and follow_up_date should be a string
        meidcation_details should have keys: 'name': string, 'dose':string, start_date:string, end_date:string, 'timings':list which is accoding to the patient eating times"""
    ),
    expected_output="Structured advice for the patient including recovery recommendations, medication details, and follow-up date.",
    agent=llm_agent
)

In [6]:
crew = Crew(agents=[llm_agent], tasks=[support_task],verbose=True)
inputs = {"general_information": general_information, "conversation": conversation}

In [7]:
result = crew.kickoff(inputs)

 [2024-06-28 20:29:33][DEBUG]: == Working Agent: Patient Support Agent
 [2024-06-28 20:29:33][INFO]: == Starting Task: 
        Here is patient general information: Patient Name: Sneh Age: 20 Symptoms: - High fever (99 degrees Fahrenheit) - Cold - Cough Pain Assessment: None provided Medication: Paracetamol, effective for a few hours Symptom Duration: 2 days Additional Information: None

        Here is the conversation between the patient and the doctor: 
Hi Sneh, how are you feeling today?

Hi, Doctor. I still have a high fever, cold, and cough. It's been like this for the past two days.

I see. Based on the symptoms you've described—high fever, cold, and cough—it sounds like you might have a viral infection, such as the flu or a common cold. These infections are quite common and usually resolve on their own with proper rest and medication. However, it’s important to monitor the fever and ensure it doesn’t get too high.

That makes sense. What can I do to feel better?

Continue takin

In [8]:
print(result)

{
"recovery_recommendations": [
"Continue taking Paracetamol for the fever. It's good to hear that it has been effective for you.",
"Take Cetirizine 10mg before bed for your cold symptoms.",
"Take Guaifenesin 200mg after breakfast and after dinner for the cough.",
"Make sure to drink plenty of fluids.",
"Get plenty of rest.",
"Monitor the fever and ensure it doesn’t get too high."
],
"medication_details": [
{
"name": "Paracetamol",
"dose": "500mg",
"start_date": "2024-06-28",
"end_date": "2024-06-30",
"timings": [
"8:00 AM",
"2:00 PM",
"8:00 PM",
"10:00 PM"
]
},
{
"name": "Cetirizine",
"dose": "10mg",
"start_date": "2024-06-28",
"end_date": "2024-06-30",
"timings": [
"10:00 PM"
]
},
{
"name": "Guaifenesin",
"dose": "200mg",
"start_date": "2024-06-28",
"end_date": "2024-06-30",
"timings": [
"8:00 AM",
"8:00 PM"
]
}
],
"follow_up_date": "2024-07-03"
}


In [9]:
import ast

def get_dict(your_string):
    try:
        return ast.literal_eval(your_string)
    except ValueError:
        return {}


response_dict =  get_dict(result)

In [11]:
for medicine in response_dict["medication_details"]:
    timings = medicine["timings"]
    for time in timings:
        print(time)
        start_date_datetime = datetime.strptime(medicine["start_date"], "%Y-%m-%d")
        end_date_datetime = datetime.strptime(medicine["end_date"], "%Y-%m-%d")
        frequency = (end_date_datetime - start_date_datetime).days + 1
        datetime_obj = datetime.strptime(
            f"{medicine['start_date']} {time}", "%Y-%m-%d %I:%M %p")
        print(f"adding event for {medicine['name']} at {datetime_obj.isoformat()} for {frequency} days")

8:00 AM
adding event for Paracetamol at 2024-06-28T08:00:00 for 3 days
2:00 PM
adding event for Paracetamol at 2024-06-28T14:00:00 for 3 days
8:00 PM
adding event for Paracetamol at 2024-06-28T20:00:00 for 3 days
10:00 PM
adding event for Paracetamol at 2024-06-28T22:00:00 for 3 days
10:00 PM
adding event for Cetirizine at 2024-06-28T22:00:00 for 3 days
8:00 AM
adding event for Guaifenesin at 2024-06-28T08:00:00 for 3 days
8:00 PM
adding event for Guaifenesin at 2024-06-28T20:00:00 for 3 days


In [12]:
from __future__ import print_function
import os.path
import pickle
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# If modifying these SCOPES, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/calendar']


creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('calendar', 'v3', credentials=creds)

for medicine in response_dict["medication_details"]:
    timings = medicine["timings"]
    for time in timings:
        start_date_datetime = datetime.strptime(
            medicine["start_date"], "%Y-%m-%d")
        end_date_datetime = datetime.strptime(medicine["end_date"], "%Y-%m-%d")
        frequency = (end_date_datetime - start_date_datetime).days + 1
        medicine_time_start = datetime.strptime(
            f"{medicine['start_date']} {time}", "%Y-%m-%d %I:%M %p")
        medicine_time_end = medicine_time_start.replace(
            minute=medicine_time_start.minute + 15)
        print(
            f"adding event for {medicine['name']} at {datetime_obj.isoformat()} for {frequency} days")
        # Create an event
        event = {
            'summary': f"Take your medicines ({medicine['name']})",
            'location': 'Home',
            'description': f"Take {medicine['name']}, dosage: {medicine['dose']}",
            'start': {
                'dateTime': f'{medicine_time_start.isoformat()}',
                'timeZone': 'Asia/Kolkata',
            },
            'end': {
                'dateTime': f'{medicine_time_end.isoformat()}',
                'timeZone': 'Asia/Kolkata',
            },
            'recurrence': [
                f'RRULE:FREQ=DAILY;COUNT={frequency}'
            ],
            'attendees': [
                {'email': 'panksoni2004@gmail.com'}
            ],
            'reminders': {
                'useDefault': True,
            },
        }

        event = service.events().insert(calendarId='primary', body=event).execute()
        print('Event created: %s' % (event.get('htmlLink')))

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=6116327917-6909rjmj8qpmbr0dcfncehsq0vrteamv.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A52782%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=jKdE1AxsMhYuJzJtKUHMHgFvsYezWG&access_type=offline
adding event for Paracetamol at 2024-06-28T20:00:00 for 3 days
Event created: https://www.google.com/calendar/event?eid=b3VtanV1ODJnajJ0OGo5MjYwdGlqYzBhZ2dfMjAyNDA2MjhUMDIzMDAwWiBwbXNvbmkyMDE2QG0
adding event for Paracetamol at 2024-06-28T20:00:00 for 3 days
Event created: https://www.google.com/calendar/event?eid=bXY3bHFsbmVhNnJtMjJsdGV0NjBhbDJtbmdfMjAyNDA2MjhUMDgzMDAwWiBwbXNvbmkyMDE2QG0
adding event for Paracetamol at 2024-06-28T20:00:00 for 3 days
Event created: https://www.google.com/calendar/event?eid=MjJkMHRrdnVyNnB0dXY5bnJqZTQydmtkOWtfMjAyNDA2MjhUMTQzMDAwWiBwbXNvbmkyMDE2QG0
adding event for Paracetamol at 2024-06-28T20:00:00 for 